In [5]:
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import panel as pn

# Initialize Panel extension
pn.extension('bokeh')

# Load datasets
age = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_age.xlsx")
disability = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_disability.xlsx")
economic_act = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_economic_activity.xlsx")
health = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_general_health.xlsx")
occupation = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_occupation.xlsx")

# Load geographical data
geojson_path = r"C:\Users\tom_r\Desktop\RBL_interview\datasets\Local_Authority_Districts_May_2021_UK_BFC_2022_-815591597668846243.geojson"
gdf = gpd.read_file(geojson_path)

# Simplify geometries to reduce complexity
gdf['geometry'] = gdf['geometry'].simplify(tolerance=100, preserve_topology=True)

# Prepare the disability data
total_number_vets = disability[
    (disability['UK armed forces veteran indicator (5 categories) Code'] == 1) | 
    (disability['UK armed forces veteran indicator (5 categories) Code'] == 2)
]
veterans_sum = total_number_vets.groupby('Lower tier local authorities')['Observation'].sum().reset_index()
veterans_sum.columns = ['Lower tier local authorities', 'Total Veterans']
veterans_sum.rename(columns={'Lower tier local authorities': 'LAD21NM'}, inplace=True)

# Merge with geographical data
merged_gdf = gdf.merge(veterans_sum, on='LAD21NM', how='left')
merged_gdf['Total Veterans'] = merged_gdf['Total Veterans'].fillna(0).astype(int)
merged_gdf.rename(columns={'LAD21NM': 'Area'}, inplace=True)

# Convert to EPSG:4326 (WGS84)
gdf = merged_gdf.to_crs(epsg=4326)

# Function to filter data
def filter_data(dataset, selected_names, selected_indicators, by_column, selected_by_values):
    data = dataset[dataset['Lower tier local authorities'].isin(selected_names)]
    if data.empty:
        print(f"No data available for areas: {selected_names}")
        return pd.DataFrame()
    filtered_data = data[
        (data['UK armed forces veteran indicator (5 categories)'].isin(selected_indicators)) &
        (data[by_column].isin(selected_by_values)) &
        (data['UK armed forces veteran indicator (5 categories)'] != -8) & 
        (data['UK armed forces veteran indicator (5 categories)'] != 4)
    ]
    grouped_data = filtered_data.groupby(['UK armed forces veteran indicator (5 categories)', by_column])['Observation'].sum().unstack().fillna(0).reset_index()
    long_format_data = grouped_data.melt(id_vars=['UK armed forces veteran indicator (5 categories)'], var_name=by_column, value_name='Observation')
    return long_format_data

# Function to generate plots
def generate_plots(selected_names, selected_indicators, 
                   disability_by, health_by, age_by, economic_by, occupation_by,
                   disability_by_values, health_by_values, age_by_values, economic_by_values, occupation_by_values):
    
    if not selected_names:
        selected_names = list(gdf['Area'])
    if not selected_indicators:
        selected_indicators = list(disability['UK armed forces veteran indicator (5 categories)'].unique())
    
    # Construct dynamic title
    title_suffix = f"for {', '.join(selected_names[:3])}..." if len(selected_names) > 3 else f"for {', '.join(selected_names)}"

    # Disability Plot
    disability_data = filter_data(disability, selected_names, selected_indicators, disability_by, disability_by_values)
    if not disability_data.empty:
        disability_plot = disability_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=disability_by,
            stacked=False,
            rot=45,
            title=f'Disability by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        disability_plot = pn.pane.Markdown("No data available for the selected areas in the disability dataset.")
    
    # Health Plot
    health_data = filter_data(health, selected_names, selected_indicators, health_by, health_by_values)
    if not health_data.empty:
        health_plot = health_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=health_by,
            stacked=False,
            rot=45,
            title=f'Health by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        health_plot = pn.pane.Markdown("No data available for the selected areas in the health dataset.")
    
    # Age Plot
    age_data = filter_data(age, selected_names, selected_indicators, age_by, age_by_values)
    if not age_data.empty:
        age_plot = age_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=age_by,
            stacked=False,
            rot=45,
            title=f'Age by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        age_plot = pn.pane.Markdown("No data available for the selected areas in the age dataset.")
    
    # Economic Activity Plot
    economic_data = filter_data(economic_act, selected_names, selected_indicators, economic_by, economic_by_values)
    if not economic_data.empty:
        economic_plot = economic_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=economic_by,
            stacked=False,
            rot=45,
            title=f'Economic Activity by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        economic_plot = pn.pane.Markdown("No data available for the selected areas in the economic activity dataset.")
    
    # Occupation Plot
    occupation_data = filter_data(occupation, selected_names, selected_indicators, occupation_by, occupation_by_values)
    if not occupation_data.empty:
        occupation_plot = occupation_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=occupation_by,
            stacked=False,
            rot=45,
            title=f'Occupation by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        occupation_plot = pn.pane.Markdown("No data available for the selected areas in the occupation dataset.")
    
    return pn.Tabs(
        ("Disability", disability_plot),
        ("Health", health_plot),
        ("Age", age_plot),
        ("Economic Activity", economic_plot),
        ("Occupation", occupation_plot)
    )

# Widgets for selecting areas and veteran indicators
area_select = pn.widgets.MultiSelect(
    name='Select Areas',
    options=list(gdf['Area'].unique()),
    value=list(gdf['Area'].unique()),  # Select all areas by default
    size=10
)

veteran_indicator_options = list(disability['UK armed forces veteran indicator (5 categories)'].unique())
veteran_indicator_select = pn.widgets.MultiSelect(
    name='Select Veteran Indicator(s)',
    options=veteran_indicator_options,
    value=veteran_indicator_options,  # Select all indicators by default
    size=10
)

# 'By' value selectors for each plot
# disability_by_select = pn.widgets.Select(
#     name='Disability By',
#     options=list(disability.columns),
#     value='Disability - Equality act disabled (4 categories)'
# )

# health_by_select = pn.widgets.Select(
#     name='Health By',
#     options=list(health.columns),
#     value='General health (3 categories)'
# )

# age_by_select = pn.widgets.Select(
#     name='Age By',
#     options=list(age.columns),
#     value='Age (B) (14 categories)'
# )

# economic_by_select = pn.widgets.Select(
#     name='Economic Activity By',
#     options=list(economic_act.columns),
#     value='Economic activity status (4 categories)'
# )

# occupation_by_select = pn.widgets.Select(
#     name='Occupation By',
#     options=list(occupation.columns),
#     value='Occupation (current) (10 categories)'
# )

# MultiSelects for filtering the 'By' values
def get_unique_values(df, column):
    return sorted(df[column].unique().tolist())

disability_by_values_select = pn.widgets.MultiSelect(
    name='Select Disability Values',
    options=get_unique_values(disability, 'Disability - Equality act disabled (4 categories)'),
    value=get_unique_values(disability, 'Disability - Equality act disabled (4 categories)'),
    size=10
)

health_by_values_select = pn.widgets.MultiSelect(
    name='Select Health Values',
    options=get_unique_values(health, 'General health (3 categories)'),
    value=get_unique_values(health, 'General health (3 categories)'),
    size=10
)

age_by_values_select = pn.widgets.MultiSelect(
    name='Select Age Values',
    options=get_unique_values(age, 'Age (B) (14 categories)'),
    value=get_unique_values(age, 'Age (B) (14 categories)'),
    size=10
)

economic_by_values_select = pn.widgets.MultiSelect(
    name='Select Economic Activity Values',
    options=get_unique_values(economic_act, 'Economic activity status (4 categories)'),
    value=get_unique_values(economic_act, 'Economic activity status (4 categories)'),
    size=10
)

occupation_by_values_select = pn.widgets.MultiSelect(
    name='Select Occupation Values',
    options=get_unique_values(occupation, 'Occupation (current) (10 categories)'),
    value=get_unique_values(occupation, 'Occupation (current) (10 categories)'),
    size=10
)

# Function to update MultiSelect options when 'By' value changes
def update_options(event, dataset, select_widget):
    options = get_unique_values(dataset, event.new)
    select_widget.options = options
    select_widget.value = options

# disability_by_select.param.watch(lambda event: update_options(event, disability, disability_by_values_select), 'value')
# health_by_select.param.watch(lambda event: update_options(event, health, health_by_values_select), 'value')
# age_by_select.param.watch(lambda event: update_options(event, age, age_by_values_select), 'value')
# economic_by_select.param.watch(lambda event: update_options(event, economic_act, economic_by_values_select), 'value')
# occupation_by_select.param.watch(lambda event: update_options(event, occupation, occupation_by_values_select), 'value')

# Bind the plot generation to the area, indicator, and 'by' value selections
plot_tabs = pn.bind(
    generate_plots,
    area_select,
    veteran_indicator_select,
    'Disability - Equality act disabled (4 categories)',
    'General health (3 categories)',
    'Age (B) (14 categories)',
    'Economic activity status (4 categories)',
    'Occupation (current) (10 categories)',
    disability_by_values_select,
    health_by_values_select,
    age_by_values_select,
    economic_by_values_select,
    occupation_by_values_select
)

# Create map plot
initial_map_plot = gdf.hvplot.polygons(
    geo=True,
    tiles='CartoLight',
    frame_height=600,
    frame_width=450,
    color='Total Veterans',
    cmap='Viridis',
    line_color='black',
    hover_cols=['Area', 'Total Veterans'],
    title="Total Veterans in UK Areas As Estimated by Census Data"
)

# Panel for the map
map_panel = pn.panel(initial_map_plot)

# Sidebar with RBL logo and Select Areas on top, Map below
sidebar = pn.Column(
    pn.Row(
        pn.pane.PNG('Royal_British_Legion_logo.png', width=300, margin=(0, 3, 0, 0)),
        pn.Column("## Select Area(s)", area_select, sizing_mode='stretch_width')
    ),
    map_panel
)

# Dashboard layout with veteran indicator selector on the right of the plots and 'by' selectors for each plot
dashboard = pn.template.FastListTemplate(
    title='Interactive Dashboard',
    sidebar_width=750,
    sidebar=sidebar,
    main=[
        pn.Row(
            plot_tabs,
            pn.Column(
                "## Select Veteran Indicator(s)",
                veteran_indicator_select,
                "## Select Disability Values",
                pn.Row(disability_by_values_select),
                "## Select Health Values",
                pn.Row(health_by_values_select),
                "## Select Age Values",
                pn.Row(age_by_values_select),
                "## Select Economic Activity Values",
                pn.Row(economic_by_values_select),
                "## Select Occupation Values",
                pn.Row(occupation_by_values_select),
                width=350, sizing_mode='stretch_height'
            )
        )
    ],
    accent_base_color="#004b87",  # RBL Blue
)

# Serve the dashboard in a new browser window
pn.serve(dashboard, start=True, show=True)


Launching server at http://localhost:51760


In [ ]:
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import panel as pn
import holoviews as hv


# Initialize Panel extension
pn.extension('bokeh', 'tabulator')

# Load datasets
age = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_age.xlsx")
disability = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_disability.xlsx")
economic_act = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_economic_activity.xlsx")
health = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_general_health.xlsx")
occupation = pd.read_excel(r"C:\Users\tom_r\Desktop\RBL_interview\datasets\veterans_by_occupation.xlsx")

# Load geographical data
geojson_path = r"C:\Users\tom_r\Desktop\RBL_interview\datasets\Local_Authority_Districts_May_2021_UK_BFC_2022_-815591597668846243.geojson"
gdf = gpd.read_file(geojson_path)

# Simplify geometries to reduce complexity
gdf['geometry'] = gdf['geometry'].simplify(tolerance=100, preserve_topology=True)

# Prepare the disability data
total_number_vets = disability[
    (disability['UK armed forces veteran indicator (5 categories) Code'] == 1) | 
    (disability['UK armed forces veteran indicator (5 categories) Code'] == 2)
]
veterans_sum = total_number_vets.groupby('Lower tier local authorities')['Observation'].sum().reset_index()
veterans_sum.columns = ['Lower tier local authorities', 'Total Veterans']
veterans_sum.rename(columns={'Lower tier local authorities': 'LAD21NM'}, inplace=True)

# Merge with geographical data
merged_gdf = gdf.merge(veterans_sum, on='LAD21NM', how='left')
merged_gdf['Total Veterans'] = merged_gdf['Total Veterans'].fillna(0).astype(int)
merged_gdf.rename(columns={'LAD21NM': 'Area'}, inplace=True)

# Convert to EPSG:4326 (WGS84)
gdf = merged_gdf.to_crs(epsg=4326)

# Function to filter data
def filter_data(dataset, selected_names, selected_indicators, by_column, selected_by_values):
    data = dataset[dataset['Lower tier local authorities'].isin(selected_names)]
    if data.empty:
        print(f"No data available for areas: {selected_names}")
        return pd.DataFrame()
    filtered_data = data[
        (data['UK armed forces veteran indicator (5 categories)'].isin(selected_indicators)) &
        (data[by_column].isin(selected_by_values)) &
        (data['UK armed forces veteran indicator (5 categories)'] != -8) & 
        (data['UK armed forces veteran indicator (5 categories)'] != 4)
    ]
    grouped_data = filtered_data.groupby(['UK armed forces veteran indicator (5 categories)', by_column])['Observation'].sum().unstack().fillna(0).reset_index()
    long_format_data = grouped_data.melt(id_vars=['UK armed forces veteran indicator (5 categories)'], var_name=by_column, value_name='Observation')
    return long_format_data

# Function to generate plots
def generate_plots(selected_names, selected_indicators, 
                   disability_by, health_by, age_by, economic_by, occupation_by,
                   disability_by_values, health_by_values, age_by_values, economic_by_values, occupation_by_values):
    
    if not selected_names:
        selected_names = list(gdf['Area'])
    if not selected_indicators:
        selected_indicators = list(disability['UK armed forces veteran indicator (5 categories)'].unique())
    
    # Construct dynamic title
    title_suffix = f"for {', '.join(selected_names[:3])}..." if len(selected_names) > 3 else f"for {', '.join(selected_names)}"

    # Disability Plot
    disability_data = filter_data(disability, selected_names, selected_indicators, disability_by, disability_by_values)
    if not disability_data.empty:
        disability_plot = disability_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=disability_by,
            stacked=False,
            rot=45,
            title=f'Disability by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        disability_plot = pn.pane.Markdown("No data available for the selected areas in the disability dataset.")
    
    # Health Plot
    health_data = filter_data(health, selected_names, selected_indicators, health_by, health_by_values)
    if not health_data.empty:
        health_plot = health_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=health_by,
            stacked=False,
            rot=45,
            title=f'Health by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        health_plot = pn.pane.Markdown("No data available for the selected areas in the health dataset.")
    
    # Age Plot
    age_data = filter_data(age, selected_names, selected_indicators, age_by, age_by_values)
    if not age_data.empty:
        age_plot = age_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=age_by,
            stacked=False,
            rot=45,
            title=f'Age by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        age_plot = pn.pane.Markdown("No data available for the selected areas in the age dataset.")
    
    # Economic Activity Plot
    economic_data = filter_data(economic_act, selected_names, selected_indicators, economic_by, economic_by_values)
    if not economic_data.empty:
        economic_plot = economic_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=economic_by,
            stacked=False,
            rot=45,
            title=f'Economic Activity by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        economic_plot = pn.pane.Markdown("No data available for the selected areas in the economic activity dataset.")
    
    # Occupation Plot
    occupation_data = filter_data(occupation, selected_names, selected_indicators, occupation_by, occupation_by_values)
    if not occupation_data.empty:
        occupation_plot = occupation_data.hvplot.bar(
            x='UK armed forces veteran indicator (5 categories)',
            y='Observation',
            by=occupation_by,
            stacked=False,
            rot=45,
            title=f'Occupation by UK Armed Forces Veteran Indicator {title_suffix}',
            ylabel='Observation',
            xlabel='UK Armed Forces Veteran Indicator',
            height=800,
            width=800,
            grid=True,
            legend='top_right'
        )
    else:
        occupation_plot = pn.pane.Markdown("No data available for the selected areas in the occupation dataset.")
    
    return pn.Tabs(
        ("Disability", disability_plot),
        ("Health", health_plot),
        ("Age", age_plot),
        ("Economic Activity", economic_plot),
        ("Occupation", occupation_plot)
    )

# Widgets for selecting areas and veteran indicators
area_select = pn.widgets.MultiSelect(
    name='Select Areas',
    options=list(gdf['Area'].unique()),
    value=list(gdf['Area'].unique()),  # Select all areas by default
    size=10
)

veteran_indicator_options = list(disability['UK armed forces veteran indicator (5 categories)'].unique())
veteran_indicator_select = pn.widgets.MultiSelect(
    name='Select Veteran Indicator(s)',
    options=veteran_indicator_options,
    value=veteran_indicator_options,  # Select all indicators by default
    size=10
)

# MultiSelects for filtering the 'By' values
def get_unique_values(df, column):
    return sorted(df[column].unique().tolist())

disability_by_values_select = pn.widgets.MultiSelect(
    name='Select Disability Values',
    options=get_unique_values(disability, 'Disability - Equality act disabled (4 categories)'),
    value=get_unique_values(disability, 'Disability - Equality act disabled (4 categories)'),
    size=10
)

health_by_values_select = pn.widgets.MultiSelect(
    name='Select Health Values',
    options=get_unique_values(health, 'General health (3 categories)'),
    value=get_unique_values(health, 'General health (3 categories)'),
    size=10
)

age_by_values_select = pn.widgets.MultiSelect(
    name='Select Age Values',
    options=get_unique_values(age, 'Age (B) (14 categories)'),
    value=get_unique_values(age, 'Age (B) (14 categories)'),
    size=10
)

economic_by_values_select = pn.widgets.MultiSelect(
    name='Select Economic Activity Values',
    options=get_unique_values(economic_act, 'Economic activity status (4 categories)'),
    value=get_unique_values(economic_act, 'Economic activity status (4 categories)'),
    size=10
)

occupation_by_values_select = pn.widgets.MultiSelect(
    name='Select Occupation Values',
    options=get_unique_values(occupation, 'Occupation (current) (10 categories)'),
    value=get_unique_values(occupation, 'Occupation (current) (10 categories)'),
    size=10
)

# Function to update MultiSelect options when 'By' value changes
def update_options(event, dataset, select_widget):
    options = get_unique_values(dataset, event.new)
    select_widget.options = options
    select_widget.value = options

# Bind the plot generation to the area, indicator, and 'by' value selections
plot_tabs = pn.bind(
    generate_plots,
    area_select,
    veteran_indicator_select,
    'Disability - Equality act disabled (4 categories)',
    'General health (3 categories)',
    'Age (B) (14 categories)',
    'Economic activity status (4 categories)',
    'Occupation (current) (10 categories)',
    disability_by_values_select,
    health_by_values_select,
    age_by_values_select,
    economic_by_values_select,
    occupation_by_values_select
)

# Create map plot with dynamic interaction
initial_map_plot = gdf.hvplot.polygons(
    geo=True,
    tiles='CartoLight',
    frame_height=600,
    frame_width=450,
    color='Total Veterans',
    cmap='Viridis',
    line_color='black',
    hover_cols=['Area', 'Total Veterans'],
    title="Total Veterans in UK Areas As Estimated by Census Data",
    tools=['tap']
)

# Create a stream to handle tap events
tap_stream = hv.streams.Tap(source=initial_map_plot, x=None, y=None)

# Define a callback to update plots based on tap
def update_selection(x, y):
    if x is None or y is None:
        return area_select.value  # Return current selection if no tap
    nearest_point = gdf.sindex.nearest((x, y, x, y), return_distance=True)
    if nearest_point[1][0] > 0.01:  # Threshold distance to find a nearby point
        return area_select.value  # Return current selection if no nearby point
    selected_area = gdf.iloc[nearest_point[0]]['Area']
    return [selected_area]

tap_stream.add_subscriber(lambda x, y: area_select.value.update(update_selection(x, y)))

# Panel for the map
map_panel = pn.panel(initial_map_plot)

# Sidebar with RBL logo and Select Areas on top, Map below
sidebar = pn.Column(
    pn.Row(
        pn.pane.PNG('Royal_British_Legion_logo.png', width=300, margin=(0, 3, 0, 0)),
        pn.Column("## Select Area(s)", area_select, sizing_mode='stretch_width')
    ),
    map_panel
)

# Dashboard layout with veteran indicator selector on the right of the plots and 'by' selectors for each plot
dashboard = pn.template.FastListTemplate(
    title='Interactive Dashboard',
    sidebar_width=750,
    sidebar=sidebar,
    main=[
        pn.Row(
            plot_tabs,
            pn.Column(
                "## Select Veteran Indicator(s)",
                veteran_indicator_select,
                "## Select Disability Values",
                pn.Row(disability_by_values_select),
                "## Select Health Values",
                pn.Row(health_by_values_select),
                "## Select Age Values",
                pn.Row(age_by_values_select),
                "## Select Economic Activity Values",
                pn.Row(economic_by_values_select),
                "## Select Occupation Values",
                pn.Row(occupation_by_values_select),
                width=350, sizing_mode='stretch_height'
            )
        )
    ],
    accent_base_color="#004b87",  # RBL Blue
)

# Serve the dashboard in a new browser window
pn.serve(dashboard, start=True, show=True)
